# Comparison between GoiStrat and naïve solution

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

Imports

In [ ]:
import sys
import pandas as pd
import seaborn as sns
import numpy as np

from IPython.display import display
from typing import Iterable, Dict
from itertools import product
import matplotlib.pyplot as plt
from pathlib import Path

Setup

In [ ]:
src_path: str = "/home/uziel/Development/goi-strat/src"
sys.path.insert(0, src_path)

In [ ]:
from data.utils import calculate_power

In [ ]:
ROOT: Path = Path("/mnt/d/phd_data/")
MSIGDB_CATS: Iterable[str] = ("H", *[f"C{i}" for i in range(1, 9)])
DATASETS_MARKERS: Dict[str, str] = {
    "TCGA-BRCA": "BRCA1",  # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8998777/
    "TCGA-LUAD": "NKX2-1",  # https://www.nature.com/articles/nature09881
    "TCGA-THCA": "HMGA2",  # https://core.ac.uk/download/pdf/46919877.pdf#page=59
    "TCGA-UCEC": "PIK3CA",  # https://pubmed.ncbi.nlm.nih.gov/28860563/, https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3060282/
    "TCGA-LUSC": "SOX2",  # https://www.cell.com/cancer-cell/fulltext/S1535-6108(16)30436-6
    "TCGA-KIRC": "CA9",  # https://www.sciencedirect.com/science/article/abs/pii/S0959804910006982
    "TCGA-HNSC": "TP63",  # https://aacrjournals.org/mcr/article/17/6/1279/270274/Loss-of-TP63-Promotes-the-Metastasis-of-Head-and
    "TCGA-LGG": "IDH1",  # https://www.neurology.org/doi/abs/10.1212/wnl.0b013e3181f96282
    "PCTA_WCDT": "FOLH1",  # https://www.nature.com/articles/nrurol.2016.26
}
PERCENTILES: Iterable[int] = (10, 15, 20, 25, 30)
RENAME_DICT: Dict = {
    "GOI_level": "GoiStrat (*/50/*)",
    "GOI_level_10": "10 / 80 / 10",
    "GOI_level_15": "15 / 70 / 15",
    "GOI_level_20": "20 / 60 / 20",
    "GOI_level_25": "25 / 50 / 25",
    "GOI_level_30": "30 / 40 / 30",
}
SAMPLE_TYPE: str = "prim"
PALETTE_STR: str = "coolwarm"  # coolwarm or flare
EFFECT_SIZE: float = 0.5
ALPHA: float = 0.05

sns.set_theme(style="whitegrid", palette=PALETTE_STR)
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = "Ubuntu Mono"
plt.rcParams["font.monospace"] = "Ubuntu Mono"

Global variables

In [ ]:
MULTI_DATASET_PATH: Path = ROOT.joinpath("MULTI_DATASET")
MULTI_DATASET_PATH.mkdir(parents=True, exist_ok=True)

## 1. Explore dataset characteristics

### 1.1. Annotation data from all splits

In [ ]:
group_counts = dict()
for dataset, marker in DATASETS_MARKERS.items():
    data_root = ROOT.joinpath(f"{dataset}_{marker}")
    annot_df = pd.read_csv(
        data_root.joinpath("data").joinpath(f"samples_annotation_{marker}_gsva.csv"),
        index_col=0,
    )
    contrast_factor = f"{marker}_level"
    group_counts[(dataset + f" ({marker})", "GOI_level")] = (
        annot_df[contrast_factor].value_counts().to_dict()
    )
    group_counts[(dataset + f" ({marker})", "GOI_level")].update(
        {"total": len(annot_df)}
    )

    for percentile in PERCENTILES:
        annot_df = pd.read_csv(
            data_root.joinpath("data").joinpath(
                f"samples_annotation_{marker}_perc.csv"
            ),
            index_col=0,
        )
        contrast_factor = f"{marker}_level_{percentile}"
        group_counts[(dataset + f" ({marker})", f"GOI_level_{percentile}")] = (
            annot_df[contrast_factor].value_counts().to_dict()
        )
        group_counts[(dataset + f" ({marker})", f"GOI_level_{percentile}")].update(
            {"total": len(annot_df)}
        )

group_counts_df = pd.DataFrame(group_counts).T
group_counts_df.to_csv(MULTI_DATASET_PATH.joinpath("group_counts_all_df.csv"))
display(group_counts_df)

In [ ]:
# Function to apply calculate_power to each row
def calculate_row_power(row: pd.Series) -> float:
    """
    Calculate the test power for a given row using the low and high values.

    Parameters:
    - row: A pandas Series representing a row of the DataFrame.

    Returns:
    - float: The calculated power for the row.
    """
    return calculate_power(
        effect_size=EFFECT_SIZE, alpha=ALPHA, n1=row["low"], n2=row["high"]
    )


# Apply the function to each row and create a new column 'power'
group_counts_df["power"] = group_counts_df.apply(calculate_row_power, axis=1)

group_counts_df.to_csv(MULTI_DATASET_PATH.joinpath("group_counts_df_with_power.csv"))
# Display the updated DataFrame
display(group_counts_df)

In [ ]:
power_map = group_counts_df["power"].to_dict()
print(power_map)

## 2. Gather and compare differential expression results

Differential expression results between low and high groups for each dataset and splitting strategy.

In [ ]:
all_degs = dict()
all_degs_scores = dict()
for dataset, marker in DATASETS_MARKERS.items():
    contrast_factors = [
        f"{marker}_level_{percentile}" for percentile in PERCENTILES
    ] + [f"{marker}_level"]
    data_root = ROOT.joinpath(f"{dataset}_{marker}")

    for contrast_factor in contrast_factors:
        deseq_results = pd.read_csv(
            data_root.joinpath("deseq2").joinpath(
                f"sample_type_{SAMPLE_TYPE}_{contrast_factor}_"
                f"{SAMPLE_TYPE}_high+{SAMPLE_TYPE}_low_"
                f"_{SAMPLE_TYPE}_high_vs_{SAMPLE_TYPE}_low_"
                "padj_0_05_all_1_0_deseq_results_unique.csv"
            ),
            index_col=0,
        )
        contrast_factor_str = contrast_factor.replace(marker, "GOI")
        all_degs[(dataset + f" ({marker})", contrast_factor_str)] = deseq_results[
            "log2FoldChange"
        ]
        all_degs_scores[(dataset + f" ({marker})", contrast_factor_str)] = (
            deseq_results["log2FoldChange"].pipe(
                lambda x: np.sqrt(np.mean(np.power(x, 2)))
            )
        )

all_degs_df = pd.DataFrame(all_degs)

In [ ]:
all_degs_scores_df = pd.Series(all_degs_scores).unstack(level=0).transpose()
all_degs_scores_df.to_csv(MULTI_DATASET_PATH.joinpath("all_degs_scores_df.csv"))
display(all_degs_scores_df)

In [ ]:
def apply_function(df, func):
    for row_idx, row in df.iterrows():
        for col_idx, value in row.items():
            df.at[row_idx, col_idx] = value * func[(row_idx, col_idx)]
    return df


all_degs_scores_df_weighted = apply_function(all_degs_scores_df, power_map)
all_degs_scores_df_weighted.to_csv(
    MULTI_DATASET_PATH.joinpath("all_degs_scores_df_weighted.csv")
)
display(all_degs_scores_df_weighted)

In [ ]:
all_degs_scores_df_weighted_ranked = all_degs_scores_df_weighted.rank(
    axis=1, method="max", ascending=False
).astype(int)
all_degs_scores_df_weighted_ranked.to_csv(
    MULTI_DATASET_PATH.joinpath("all_degs_scores_df_weighted_ranked.csv")
)
display(all_degs_scores_df_weighted_ranked)

In [ ]:
all_degs_scores_df_weighted_ranked_summary = (
    all_degs_scores_df_weighted_ranked.median().sort_values(ascending=True)
)
all_degs_scores_df_weighted_ranked_summary.to_csv(
    MULTI_DATASET_PATH.joinpath("all_degs_scores_df_weighted_ranked_summary.csv")
)
display(all_degs_scores_df_weighted_ranked_summary)

### 2.1. Visualise results

In [ ]:
# Melt the DataFrame to long format for seaborn
all_degs_scores_df_weighted_melted = all_degs_scores_df_weighted.reset_index().melt(
    id_vars="index", var_name="Column", value_name="Score"
)

all_degs_scores_df_weighted_melted["Column"] = all_degs_scores_df_weighted_melted[
    "Column"
].map(RENAME_DICT)
all_degs_scores_df_weighted_melted["index"] = all_degs_scores_df_weighted_melted[
    "index"
].str.replace(" (", "\n(")

# Create the barplot
plt.figure(figsize=(12, 6))
barplot = sns.barplot(
    data=all_degs_scores_df_weighted_melted,
    x="index",
    y="Score",
    hue="Column",
    palette=PALETTE_STR,
)

# Customize the plot
plt.xlabel("")
plt.ylabel("Score")
plt.title(
    "Weighted Root-mean-squared LFC scores of DEGs for different GOI splitting strategies"
)

# Add a black border to the bar corresponding to "GOI_level"
for patch, level in zip(barplot.patches, all_degs_scores_df_weighted_melted["Column"]):
    if level == "GoiStrat (*/50/*)":
        patch.set_edgecolor("black")
        patch.set_linewidth(1)

# Move the legend outside of the figure
plt.legend(bbox_to_anchor=(1.02, 1), loc="upper left", borderaxespad=0.0)

# Show the plot
plt.tight_layout()

plt.savefig(
    MULTI_DATASET_PATH.joinpath("all_degs_scores_df_weighted_barplot.pdf"),
    bbox_inches="tight",
)
plt.savefig(
    MULTI_DATASET_PATH.joinpath("all_degs_scores_df_weighted_barplot.svg"),
    bbox_inches="tight",
)

plt.show()

## 3. Gather and compare differential enrichment results

Differential enrichment results between low and high groups for each dataset and splitting strategy.

In [ ]:
all_degss = dict()
all_degss_scores = dict()
for dataset, marker in DATASETS_MARKERS.items():
    contrast_factors = [
        f"{marker}_level_{percentile}" for percentile in PERCENTILES
    ] + [f"{marker}_level"]
    data_root = ROOT.joinpath(f"{dataset}_{marker}")
    msigdb_cats_meta_dfs = {
        msigdb_cat: pd.read_csv(
            ROOT.joinpath(dataset)
            .joinpath("data")
            .joinpath("gsva")
            .joinpath(f"{msigdb_cat}_meta.csv"),
            index_col=0,
        )
        for msigdb_cat in MSIGDB_CATS
    }

    for contrast_factor, msigdb_cat in product(contrast_factors, MSIGDB_CATS):
        gsva_results = pd.read_csv(
            data_root.joinpath("diff_gsva")
            .joinpath(msigdb_cat)
            .joinpath(
                f"sample_type_{SAMPLE_TYPE}_{contrast_factor}_"
                f"{SAMPLE_TYPE}_high+{SAMPLE_TYPE}_low_"
                f"_{SAMPLE_TYPE}_high_vs_{SAMPLE_TYPE}_low_"
                "top_table_padj_0_05_all_0_0.csv"
            ),
            index_col=0,
        )
        contrast_factor_str = contrast_factor.replace(marker, "GOI")
        all_degss[(dataset + f" ({marker})", contrast_factor_str, msigdb_cat)] = (
            gsva_results["log2FoldChange"]
        )
        all_degss_scores[
            (dataset + f" ({marker})", contrast_factor_str, msigdb_cat)
        ] = gsva_results["log2FoldChange"].pipe(
            lambda x: (x.count() / len(msigdb_cats_meta_dfs[msigdb_cat]))
            * np.sqrt(np.mean(np.power(x, 2)))
        )

all_degss_df = pd.DataFrame(all_degss)

In [ ]:
all_degss_scores_df = (
    pd.Series(all_degss_scores).unstack(level=[0, 1]).median().unstack()
)
all_degss_scores_df.to_csv(MULTI_DATASET_PATH.joinpath("all_degss_scores_df.csv"))
display(all_degss_scores_df)

In [ ]:
def apply_function(df, func):
    for row_idx, row in df.iterrows():
        for col_idx, value in row.items():
            df.at[row_idx, col_idx] = value * func[(row_idx, col_idx)]
    return df


all_degss_scores_df_weighted = apply_function(all_degss_scores_df, power_map)
all_degss_scores_df_weighted.to_csv(
    MULTI_DATASET_PATH.joinpath("all_degss_scores_df_weighted.csv")
)
display(all_degss_scores_df_weighted)

In [ ]:
all_degss_scores_df_weighted_ranked = all_degss_scores_df_weighted.rank(
    axis=1, method="max", ascending=False
).astype(int)
all_degss_scores_df_weighted_ranked.to_csv(
    MULTI_DATASET_PATH.joinpath("all_degss_scores_df_weighted_ranked.csv")
)
display(all_degss_scores_df_weighted_ranked)

In [ ]:
all_degss_scores_df_weighted_ranked_summary = (
    all_degss_scores_df_weighted_ranked.median().sort_values(ascending=True)
)
all_degss_scores_df_weighted_ranked_summary.to_csv(
    MULTI_DATASET_PATH.joinpath("all_degss_scores_df_weighted_ranked_summary.csv")
)
display(all_degss_scores_df_weighted_ranked_summary)

### 3.1. Visualise results

In [ ]:
# Melt the DataFrame to long format for seaborn
all_degss_scores_df_weighted_melted = all_degss_scores_df_weighted.reset_index().melt(
    id_vars="index", var_name="Column", value_name="Score"
)
all_degss_scores_df_weighted_melted["index"] = all_degss_scores_df_weighted_melted[
    "index"
].str.replace(" (", "\n(")

all_degss_scores_df_weighted_melted["Column"] = all_degss_scores_df_weighted_melted[
    "Column"
].map(RENAME_DICT)

# Create the barplot
plt.figure(figsize=(12, 6))
barplot = sns.barplot(
    data=all_degss_scores_df_weighted_melted,
    x="index",
    y="Score",
    hue="Column",
    palette=PALETTE_STR,
)

# Customize the plot
plt.xlabel("")
plt.ylabel("Score")
plt.title(
    "Weighted Median Functional Enrichment Scores for different GOI splitting strategies"
)

# Add a black border to the bar corresponding to "GOI_level"
for patch, level in zip(barplot.patches, all_degss_scores_df_weighted_melted["Column"]):
    if level == "GoiStrat (*/50/*)":
        patch.set_edgecolor("black")
        patch.set_linewidth(1)

# Move the legend outside of the figure
plt.legend(bbox_to_anchor=(1.02, 1), loc="upper left", borderaxespad=0.0)

# Show the plot
plt.tight_layout()

plt.savefig(
    MULTI_DATASET_PATH.joinpath("all_degss_scores_df_weighted_barplot.pdf"),
    bbox_inches="tight",
)
plt.savefig(
    MULTI_DATASET_PATH.joinpath("all_degss_scores_df_weighted_barplot.svg"),
    bbox_inches="tight",
)

plt.show()

- Median functional difference scores have been weighted by the power of the differential analysis test for a fairer comparison.
- We can observe a considerable heterogenity in the ability of each method to successfully split the samples such that the functional differences are maximised. In spite of this, the GoiStrat method demonstrates a higher consistency in its ability to identify functional differences between the high and low groups.
- It ranks first in 8 out of 9 datasets, and a close third in the last one.